===============================\
   Compression Pipeline\
===============================

In [1]:
!apt-get install python3.11-distutils
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.11-distutils'
python3-distutils is already the newest version (3.10.8-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
%pip install -q git+https://github.com/satabios/sconce

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:0

In [ ]:
from collections import defaultdict, OrderedDict
from sconce import sconce

import copy
import numpy as np
import torch
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchvision.datasets import *
from torchvision.transforms import *
import torch.optim as optim

assert torch.cuda.is_available(), \
"The current runtime does not have CUDA support." \
"Please go to menu bar (Runtime - Change runtime type) and select GPU"

Load the Pre-Trained Model Weights

In [4]:
!wget https://huggingface.co/satabios/pre-trained_cifar10/resolve/main/vgg.cifar.pretrained.pth?download=true -O vgg.cifar.pretrained.pth

--2025-01-27 01:11:34--  https://huggingface.co/satabios/pre-trained_cifar10/resolve/main/vgg.cifar.pretrained.pth?download=true
Resolving huggingface.co (huggingface.co)... 3.165.160.12, 3.165.160.59, 3.165.160.61, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/55/fb/55fb4781756edef0ec8cbd6e84a7a05c9478cb5c1d83647b7e7d0a7b31bf7e7d/356e478a06348b2497dbaddddbfd09a4c454d6df7c9bcdf36e767b38a93ff43a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27vgg.cifar.pretrained.pth%3B+filename%3D%22vgg.cifar.pretrained.pth%22%3B&Expires=1737943895&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNzk0Mzg5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzU1L2ZiLzU1ZmI0NzgxNzU2ZWRlZjBlYzhjYmQ2ZTg0YTdhMDVjOTQ3OGNiNWMxZDgzNjQ3YjdlN2QwYTdiMzFiZjdlN2QvMzU2ZTQ3OGEwNjM0OGIyNDk3ZGJhZGRkZGJmZDA5YTRjNDU0ZDZkZjdj

In [6]:

class VGG(nn.Module):
  ARCH = [64, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

  def __init__(self) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    in_channels = 3
    for x in self.ARCH:
      if x != 'M':
        # conv-bn-relu
        add("conv", nn.Conv2d(in_channels, x, 3, padding=1, bias=False))
        add("bn", nn.BatchNorm2d(x))
        add("relu", nn.ReLU(True))
        in_channels = x
      else:
        # maxpool
        add("pool", nn.MaxPool2d(2))

    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(512, 10)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    # backbone: [N, 3, 32, 32] => [N, 512, 2, 2]
    x = self.backbone(x)

    # avgpool: [N, 512, 2, 2] => [N, 512]
    x = x.mean([2, 3])

    # classifier: [N, 512] => [N, 10]
    x = self.classifier(x)
    return x


#load the pretrained model
model_path = "vgg.cifar.pretrained.pth"

model = VGG().cuda()
checkpoint = torch.load(model_path, weights_only=True)
model.load_state_dict(checkpoint)

<ipython-input-6-b07a58dd4246>:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


<All keys matched successfully>

Setup the Dataset

In [ ]:
image_size = 32
transforms = {
    "train": transforms.Compose([
        RandomCrop(image_size, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
    ]),
    "test": ToTensor(),
}
dataset = {}
for split in ["train", "test"]:

    dataset[split] = CIFAR10(
    root="data/cifar10",
    train=(split == "train"),
    download=True,
    transform=transforms[split],
    )

dataloader = {}
for split in ['train', 'test']:
  dataloader[split] = DataLoader(
    dataset[split],
    batch_size=512,
    shuffle=(split == 'train'),
    num_workers=0,
    pin_memory=True,
  )




sconce Configurations

**Channel-Wise Pruning**

In [8]:
sconces = sconce()
sconces.model= copy.deepcopy(model)
sconces.criterion = nn.CrossEntropyLoss() # Loss
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader
sconces.epochs = 1 #Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "vgg-cwp"
sconces.prune_mode = "CWP" # Supports Automated Pruning Ratio Detection


In [ ]:
# Compress the model Channel-Wise
sconces.compress()


Original Dense Model Size Model=35.20 MiB


Original Model Validation Accuracy: 93.13627254509018 %

 Channel-Wise Pruning


layer:  56%|█████▌    | 5/9 [03:43<03:05, 46.48s/it]

In [ ]:


sconces = sconce()
sconces.model= copy.deepcopy(model)
sconces.criterion = nn.CrossEntropyLoss() # Loss
sconces.optimizer= optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
sconces.dataloader = dataloader
sconces.epochs = 1 #Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "vgg-gmp"
sconces.prune_mode = "GMP" # Supports Automated Pruning Ratio Detection


Easy function calls for Train and Validated the Model on the given dataset

In [ ]:
# Train the model
sconces.train()
# Evaludate the model
sconces.evaluate()

Magic Happens here: Compress the model(GMP pruning is set as the prune mode[sconces.prune_mode] above)

In [ ]:
sconces.compress()

**Venum Pruning a better version of Wanda Pruning**

In [ ]:
# from sconce import sconce

# sconces = sconce()
# sconces.model = copy.deepcopy(model)
# sconces.criterion = nn.CrossEntropyLoss()  # Loss
# sconces.optimizer = optim.Adam(sconces.model.parameters(), lr=1e-4)
# sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)
# sconces.dataloader = dataloader
# sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# sconces.experiment_name = "vgg-venum"
# sconces.prune_mode = "venum"  # Supports Automated Pruning Ratio Detection
# sconces.compress()

Spiking Neural Network Compression


In [ ]:
!pip install snntorch -q

In [ ]:
# Import snntorch libraries
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt
from torch import optim

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools



In [ ]:

# Event Drive Data

# dataloader arguments
batch_size = 128
data_path = "./data/mnist"

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Define a transform
transform = transforms.Compose(
    [
        transforms.Resize((28, 28)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize((0,), (1,)),
    ]
)

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(
    mnist_train, batch_size=batch_size, shuffle=True, drop_last=True
)
test_loader = DataLoader(
    mnist_test, batch_size=batch_size, shuffle=True, drop_last=True
)


In [ ]:
from sconce import sconce
sconces = sconce()
# Set you Dataloader
dataloader = {}
dataloader["train"] = train_loader
dataloader["test"] = test_loader
sconces.dataloader = dataloader

In [ ]:
#Enable snn in sconce
sconces.snn = True

# Load your snn Model
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
snn_model = nn.Sequential(
    nn.Conv2d(1, 12, 5),
    nn.MaxPool2d(2),
    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
    nn.Conv2d(12, 64, 5),
    nn.MaxPool2d(2),
    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
    nn.Flatten(),
    nn.Linear(64 * 4 * 4, 10),
    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True),
).to('cuda')


#Load the pretrained weights
snn_pretrained_model_path = "drive/MyDrive/Efficientml/Efficientml.ai/snn_model.pth"
snn_model.load_state_dict(torch.load(snn_pretrained_model_path))  # Model Definition
sconces.model = snn_model

In [ ]:

sconces.optimizer = optim.Adam(sconces.model.parameters(), lr=1e-4)
sconces.scheduler = optim.lr_scheduler.CosineAnnealingLR(sconces.optimizer, T_max=200)

sconces.criterion = SF.ce_rate_loss()

sconces.epochs = 10  # Number of time we iterate over the data
sconces.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sconces.experiment_name = "snn-gmp"  # Define your experiment name here
sconces.prune_mode = "GMP"
sconces.num_finetune_epochs = 1


In [ ]:
sconces.compress()